In [60]:
import pandas as pd
import datetime

In [68]:
seodf = pd.read_csv("Texas.csv")
seodf

,Property Name,Trracking ID,View ID,Website Sessions,Bounce Rate,Pages/Session,Avg. Time On Site
0,2626 Fountain View,UA-50039503-1,178482103,NaN,NaN,NaN,NaN
1,Block 334,UA-50019923-1,178452865,NaN,NaN,NaN,NaN
2,Broadstone Barker Cypress,UA-50039503-15,188832488,NaN,NaN,NaN,NaN
3,Broadstone Burnet,UA-50024608-5,175521826,NaN,NaN,NaN,NaN
4,Broadstone Energy Park,UA-79862490-1,177352497,NaN,NaN,NaN,NaN
5,Broadstone Falcon Landing,UA-53195409-1,159128699,NaN,NaN,NaN,NaN
6,Broadstone Harmony,UA-74819106-1,178703323,NaN,NaN,NaN,NaN
7,Broadstone Oak Hills,UA-69125602-4,174632823,NaN,NaN,NaN,NaN
8,Broadstone Skyline,UA-67059904-1,178522043,NaN,NaN,NaN,NaN
9,Broadstone Summer Street,UA-50039503-17,195390105,NaN,NaN,NaN,NaN


In [62]:
seodf_str = seodf.astype({"View ID":str})
view_id_list = seodf_str["View ID"].tolist()
view_sample = view_id_list[0:5]
# view_sample
len(view_id_list)

15

# Loop Through List of Accounts

In [63]:
"""Hello Analytics Reporting API V4."""

import argparse

from apiclient.discovery import build
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
CLIENT_SECRETS_PATH = 'client_secrets.json' # Path to client_secrets.json file.
sessions=[]

def initialize_analyticsreporting():
  """Initializes the analyticsreporting service object.

  Returns:
    analytics an authorized analyticsreporting service object.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      CLIENT_SECRETS_PATH, scope=SCOPES,
      message=tools.message_if_missing(CLIENT_SECRETS_PATH))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage('analyticsreporting.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)

  return analytics

df_list = []

start_date = "2019-04-01"
end_date = "2019-04-30"
    
def get_report(analytics):
    print(f'View ID is {VIEW_ID}')
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
    return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
          'metrics': [{'expression': 'ga:sessions'},{'expression': 'ga:bounceRate'}, {'expression': 'ga:pageviewsPerSession'}, {'expression': 'ga:avgSessionDuration'}]
#           'dimensions':[{'name': 'ga:channelGrouping'}]   
        }]
      }
    ).execute()

def print_response(response):
    """Parses and prints the Analytics Reporting API V4 response"""


    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
        #     create empty list to later append dictionaries into. 

        for row in rows:
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])
    #         Create empty dictionary which will contain key,value pairs for relevant metrics
            mt_dict = dict({})
            mt_dict["View ID"] = VIEW_ID
            mt_dict["Start Date"] = start_date
            mt_dict["End Date"] = end_date
            for header, dimension in zip(dimensionHeaders, dimensions):
                print (header + ': ' + dimension)
    #             Create an entry in the dictionary with the Dimension as the Key (splitting for formatting) and the dimension value as the value
                mt_dict[header.split(":")[-1]] = dimension
            for i, values in enumerate(dateRangeValues):
                print ('Date range (' + str(i) + ')')
                for metricHeader, value in zip(metricHeaders, values.get('values')):
                    print (metricHeader.get('name') + ': ' + value)
    #                 Create an entry in the dictionary with the Metric as the key and the metric value as the value
                    mt_dict[metricHeader.get('name').split(":")[-1]] = value

    # Append dictionary for each iteration into list
            df_list.append(mt_dict)

    # Return a DataFrame object that is created from the above list
    return pd.DataFrame(df_list)



def main():

    analytics = initialize_analyticsreporting()
    response = get_report(analytics)
    df = print_response(response)
    return df

for VIEW_ID in view_id_list:
    main()

# Include below if using this in a .py File
# if __name__ == '__main__':
#   main()

View ID is 178482103
Date range (0)
ga:sessions: 5000
ga:bounceRate: 36.720000000000006
ga:pageviewsPerSession: 2.7714
ga:avgSessionDuration: 124.915
View ID is 178452865
Date range (0)
ga:sessions: 5265
ga:bounceRate: 42.83000949667616
ga:pageviewsPerSession: 2.6938271604938273
ga:avgSessionDuration: 125.04102564102564
View ID is 188832488
Date range (0)
ga:sessions: 1986
ga:bounceRate: 89.82880161127895
ga:pageviewsPerSession: 1.4154078549848943
ga:avgSessionDuration: 58.65508559919436
View ID is 175521826
Date range (0)
ga:sessions: 6112
ga:bounceRate: 41.16492146596858
ga:pageviewsPerSession: 2.7655431937172774
ga:avgSessionDuration: 134.85225785340313
View ID is 177352497
Date range (0)
ga:sessions: 6807
ga:bounceRate: 48.09754664316145
ga:pageviewsPerSession: 2.8135742617893347
ga:avgSessionDuration: 145.9701777581901
View ID is 159128699
Date range (0)
ga:sessions: 4733
ga:bounceRate: 30.065497570251427
ga:pageviewsPerSession: 3.925839847876611
ga:avgSessionDuration: 161.2319881

In [69]:
len(df_list)

# Have a default dictionary that has an empty dictionary in it in order to use the View ID KEY Value pair as the KEY.

13

# Writing Output to DataFrame

In [70]:
result_df = pd.DataFrame.from_dict(df_list)
result_df["View ID"] = pd.to_numeric(result_df["View ID"])
result_df["sessions"] = pd.to_numeric(result_df["sessions"])
result_df["bounceRate"] = pd.to_numeric(result_df["bounceRate"])
result_df["pageviewsPerSession"] = pd.to_numeric(result_df["pageviewsPerSession"])
result_df["avgSessionDuration"] = pd.to_numeric(result_df["avgSessionDuration"])
result_df.head()
# result_df.dtypes

,End Date,Start Date,View ID,avgSessionDuration,bounceRate,pageviewsPerSession,sessions
0,2019-04-30,2019-04-01,178482103,124.915000,36.720000,2.771400,5000
1,2019-04-30,2019-04-01,178452865,125.041026,42.830009,2.693827,5265
2,2019-04-30,2019-04-01,188832488,58.655086,89.828802,1.415408,1986
3,2019-04-30,2019-04-01,175521826,134.852258,41.164921,2.765543,6112
4,2019-04-30,2019-04-01,177352497,145.970178,48.097547,2.813574,6807


# Merge DataFrames


In [73]:
merged = seodf.merge(result_df,how="inner", on="View ID")
merged = merged[['Property Name','Start Date', 'End Date','View ID', 'sessions', 'bounceRate','pageviewsPerSession','avgSessionDuration']]

merged = merged.rename(columns={"sessions": "Website Sessions",
                               "bounceRate": "Bounce Rate",
                               "pageviewsPerSession": "Pages/Session",
                               "avgSessionDuration": "Avg_Time_On_Site"})

merged["Bounce Rate"] = pd.Series([round(val,2) for val in merged["Bounce Rate"]], index = merged.index)
merged["Pages/Session"] = pd.Series([round(val,2) for val in merged["Pages/Session"]], index = merged.index)
merged["Bounce Rate"] = pd.Series(["{0:.2f}%".format(val * 1) for val in merged["Bounce Rate"]], index = merged.index)
merged['Avg_Time_On_Site'] = pd.to_datetime(merged["Avg_Time_On_Site"], unit='m')
merged['Avg_Time_On_Site'] = merged['Avg_Time_On_Site'].astype(str)
merged['Avg_Time_On_Site'] = (merged.Avg_Time_On_Site.str.split(' ').str[1])
merged['Avg_Time_On_Site'] = (merged.Avg_Time_On_Site.str.split('.').str[0])
merged['Avg_Time_On_Site'] = (merged.Avg_Time_On_Site.str.rsplit(':',1).str[0])

merged = merged.rename(columns={'Avg_Time_On_Site': 'Avg. Time On Site'})
merged.reset_index(drop=True,inplace=True)
merged.head()

,Property Name,Start Date,End Date,View ID,Website Sessions,Bounce Rate,Pages/Session,Avg. Time On Site
0,2626 Fountain View,2019-04-01,2019-04-30,178482103,5000,36.72%,2.77,02:04
1,Block 334,2019-04-01,2019-04-30,178452865,5265,42.83%,2.69,02:05
2,Broadstone Barker Cypress,2019-04-01,2019-04-30,188832488,1986,89.83%,1.42,00:58
3,Broadstone Burnet,2019-04-01,2019-04-30,175521826,6112,41.16%,2.77,02:14
4,Broadstone Energy Park,2019-04-01,2019-04-30,177352497,6807,48.10%,2.81,02:25


In [54]:
merged.to_csv("Reports/Cristine's_Broadstones.csv")